In [13]:
import psycopg2
import pandas as pd
from config import config

In [14]:
used_car_file_path = "Resources/UsedCars.zip"
fuel_economy_path = "Resources/real-world-fuel-efficiency.csv"

used_cars_metadata = pd.read_csv(used_car_file_path)
fuel_economy_metadata = pd.read_csv(fuel_economy_path)

used_car_df     = pd.DataFrame(used_cars_metadata)
fuel_economy_df = pd.DataFrame(fuel_economy_metadata)

fuel_economy_df.columns

C:\Users\aslig\anaconda3\envs\PythonData\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (20) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Index(['YEAR', 'MAKE', 'MODEL', 'Hybrid/Non-Hybrid', 'STANDARD TYPE',
       'EPA RATING (CITY)', 'Vehicle Count', 'TOTAL ACTUAL MILES',
       'TOTAL ACTUAL FUEL', 'EPA EXPECTED FUEL', 'ACTUAL FUEL ECONOMY Geotab',
       'PERCENT % DIFFERENCE ACTUAL TO EPA', 'ESTIMATED FUEL COSTS PER GALLON',
       'FUEL COSTS PER MILE'],
      dtype='object')

In [15]:
fuel_economy_df.drop(columns=['Vehicle Count'],inplace=True)

In [3]:
used_car_df.head(2)

,id,vin,price,miles,stock_no,year,make,model,trim,body_type,...,drivetrain,transmission,fuel_type,engine_size,engine_block,seller_name,street,city,state,zip
0,38b2f52e-8f5d,1GCWGFCF3F1284719,20998.0,115879.0,W1T503168C,2015,Chevrolet,Express Cargo,Work Van,Cargo Van,...,RWD,Automatic,E85 / Unleaded,4.8,V,nissan ellicott city,8569 Baltimore National Pike,Ellicott City,MD,21043
1,84327e45-6cb6,1GCPTEE15K1291189,52997.0,28568.0,9U2Y425A,2019,Chevrolet,Colorado,ZR2,Pickup,...,4WD,Automatic,Diesel,2.8,I,young kia,308 North Main Street,Layton,UT,84041


In [4]:
print(used_car_df['vehicle_type'].unique())

['Truck' 'Car' nan]


In [5]:
#drop unessecessary columns
used_car_df.drop(columns=['vin', 'seller_name', 'street','city','state','zip','stock_no'],inplace=True)

In [6]:
len(used_car_df.columns)

14

In [7]:
conn = psycopg2.connect("dbname=Cars user=postgres password=postgres")

In [8]:
print("Number of nulls in each column")
print(used_car_df.isnull().sum(axis = 0))

Number of nulls in each column
id                  0
price           39846
miles            3294
year                0
make                0
model             529
trim             1029
body_type        1861
vehicle_type     3675
drivetrain       1000
transmission      906
fuel_type        3070
engine_size      8493
engine_block     8518
dtype: int64


In [9]:
price_df = used_car_df[{'price','year','make','model'}]

In [10]:
print(price_df)

             make  year    price          model
0       Chevrolet  2015  20998.0  Express Cargo
1       Chevrolet  2019  52997.0       Colorado
2       Chevrolet  2010   6500.0            HHR
3       Chevrolet  2016  23024.0       Colorado
4       Chevrolet  2010      NaN         Impala
...           ...   ...      ...            ...
424759     Nissan  2018  20975.0         Altima
424760     Nissan  2011   9900.0         Altima
424761     Nissan  2018  18495.0         Altima
424762     Nissan  2019  23496.0         Altima
424763     Nissan  2015      NaN         Altima

[424764 rows x 4 columns]


In [11]:
used_car_df.drop(columns=['price'],inplace=True)

In [12]:
price_df.set_index(['make','model','year'],inplace=True)
price_df.head(5)

price
make      model         year         
Chevrolet Express Cargo 2015  20998.0
          Colorado      2019  52997.0
          HHR           2010   6500.0
          Colorado      2016  23024.0
          Impala        2010      NaN

In [ ]:
print(type(conn))

In [ ]:
def connect():
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # read connection parameters
        params = config()

        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params)
		
        # create a cursor
        cur = conn.cursor()
        
	# execute a statement
        print('PostgreSQL database version:')
        cur.execute('SELECT version()')

        # display the PostgreSQL database server version
        db_version = cur.fetchone()
        print(db_version)
       
	# close the communication with the PostgreSQL
        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(f"The error is {error}")
    else:
        print("There was no error")
    finally:
        if conn is not None:
            conn.close()
            print('Database connection closed.')


if __name__ == '__main__':
    connect()


In [ ]:
def create_tables():
    """ create tables in the PostgreSQL database"""
    commands = (
        """
        CREATE TABLE vendors (
            vendor_id SERIAL PRIMARY KEY,
            vendor_name VARCHAR(255) NOT NULL
        )
        """,
        """ CREATE TABLE parts (
                part_id SERIAL PRIMARY KEY,
                part_name VARCHAR(255) NOT NULL
                )
        """,
        """
        CREATE TABLE part_drawings (
                part_id INTEGER PRIMARY KEY,
                file_extension VARCHAR(5) NOT NULL,
                drawing_data BYTEA NOT NULL,
                FOREIGN KEY (part_id)
                REFERENCES parts (part_id)
                ON UPDATE CASCADE ON DELETE CASCADE
        )
        """,
        """
        CREATE TABLE vendor_parts (
                vendor_id INTEGER NOT NULL,
                part_id INTEGER NOT NULL,
                PRIMARY KEY (vendor_id , part_id),
                FOREIGN KEY (vendor_id)
                    REFERENCES vendors (vendor_id)
                    ON UPDATE CASCADE ON DELETE CASCADE,
                FOREIGN KEY (part_id)
                    REFERENCES parts (part_id)
                    ON UPDATE CASCADE ON DELETE CASCADE
        )
        """)
    conn = None
    try:
        # read the connection parameters
        params = config()
        # connect to the PostgreSQL server
        conn = psycopg2.connect(**params)
        cur = conn.cursor()
        # create table one by one
        for command in commands:
            cur.execute(command)
        # close communication with the PostgreSQL database server
        cur.close()
        # commit the changes
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()


if __name__ == '__main__':
    create_tables()

In [19]:
print(conn)

<connection object at 0x0000012B004C0370; dsn: 'user=postgres password=xxx dbname=Cars', closed: 0>


In [20]:
#rds_connection_string = "<insert user name>:<insert password>@localhost:5432/customer_db"
#engine = create_engine(f'postgresql://{rds_connection_string}')

used_car_df.to_sql(name='used_cars', con=conn, if_exists='append', index=False)







DatabaseError: Execution failed on sql 'SELECT name FROM sqlite_master WHERE type='table' AND name=?;': relation "sqlite_master" does not exist
LINE 1: SELECT name FROM sqlite_master WHERE type='table' AND name=?...
                         ^
